### **NLP using Tensorflow**

In [1]:
# DL needs
import tensorflow as tf
import keras as kr

# Data needs
import pandas as pd
from sklearn.model_selection import train_test_split

# Numerical computation needs
import numpy as np

# plotting needs
import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# ensuring reproducibility
random_seed=42
tf.random.set_seed(random_seed)
import sys

sys.path.append('/home/rudraksha14/Desktop/RAY_RISE_ABOVE_YOURSELF/Programming/tensorflow')
import important_functionalities as impf

2025-03-24 10:57:45.941670: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
baseline_results = {'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

model_1_results = {'accuracy': 80.4461942257218,
 'precision': 0.8065100939758145,
 'recall': 0.8044619422572179,
 'f1_score': 0.8028505735911119}

model_2_results = {'accuracy': 74.80314960629921,
 'precision': 0.7475572440372034,
 'recall': 0.7480314960629921,
 'f1_score': 0.7475400591051667}

model_3_results = {'accuracy': 75.45931758530183,
 'precision': 0.7546642240189775,
 'recall': 0.7545931758530183,
 'f1_score': 0.7531886844350836}

In [3]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
# shuffle training dataframe
train_df_shuffled=train_df.sample(frac=1,random_state=random_seed) # frac: percentage of  data to be shuffled
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [5]:
from sklearn.model_selection import train_test_split

train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled['text'].to_numpy(),train_df_shuffled['target'].to_numpy(),test_size=0.1,random_state=random_seed)

In [6]:
# finding the average number of words/tokens in the training tweets
avg_len=round(sum([len(sentence.split()) for sentence in train_sentences])/len(train_sentences))
print(avg_len)
# setup text vectorization variables
max_vocab_length = 10000 # get the most common 10k words to have in our vocab 
max_len = avg_len # max length of our sequence (eg. how many words from the tweet does our model see?)

# creating the text_vectorizer
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_vocab_length,  # how many words in vocabulary, if None there is no cap on the vocab, it automatically adds <OOV> (Out Of Vocabulary) / Unknown
    output_mode='int',  # how to map tokens/words to numbers
    output_sequence_length=max_len,  # None --> sets each sequence to longest sequence
)

15


In [7]:
# mapping the text vectorization layer to text data and turning it into numbers
text_vectorizer.adapt(train_sentences)

In [8]:
# creating an embedding
embedding = tf.keras.layers.Embedding(input_dim = max_vocab_length,
                                      output_dim = 128,
                                      input_length = max_len,
                                      # embedding_initializer = 'uniform' # default:uniform random nos
                                    )
embedding

/home/rudraksha14/.local/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Embedding name=embedding, built=False>

**13. Model 4: [Bidirectional LSTM (RNN)]**

**Note:**
* Normal RNN's go from left to right (eg: reading english), while bidirectional RNN goes from left to right as well as right to left.
* The bidirectional wrapper works for both LSTM as well as GRU
* Expects 3D input [batch, timesteps/sequences, feature] {each sequence is treated as a time step}
* Inorder to stack recurrent layers on top of each other, we need to set `return_sequences=True`
* Good for sequences which may benifit from passing forward and backwards (eg. translation or longer passages or text)

In [24]:
inputs=tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)

x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64,return_sequences=True))(x) # returns representation of sequence in form of a sequence
x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=64))(x)
x = tf.keras.layers.Dense(units=64,activation='relu')(x)
outputs=tf.keras.layers.Dense(units=1,activation='sigmoid')(x)
model_4 = tf.keras.models.Model(inputs,outputs,name='model_4')

In [13]:
# get a summary
model_4.summary()

Model: "model_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization              │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 15, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 128)            │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,461,633 (5.58 MB)

 Trainable params: 1,461,633 (5.58 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# compile the model
model_4.compile(loss='binary_crossentropy', 
                metrics=['accuracy'], 
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [15]:
# create a tensorboard callback (need to create a new one for each model)
SAVE_DIR = 'model_logs'

# fit the model
history_model_4 = model_4.fit(train_sentences,train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[impf.create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name='model_4_Bidirectional')]
                              )

Saving TensorBoard log files to : model_logs/model_4_Bidirectional/20250324-110706
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step - accuracy: 0.6777 - loss: 0.5780 - val_accuracy: 0.7940 - val_loss: 0.4586
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.8644 - loss: 0.3297 - val_accuracy: 0.7467 - val_loss: 0.5459
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.9182 - loss: 0.2060 - val_accuracy: 0.7507 - val_loss: 0.6261
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.9615 - loss: 0.1093 - val_accuracy: 0.7664 - val_loss: 0.7971
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.9686 - loss: 0.0905 - val_accuracy: 0.7493 - val_loss: 1.0599


In [16]:
# making predictions:
model_4_preds=model_4.predict(val_sentences)
thresh=0.5
model_4_preds_threshed=list(map(lambda x: 1 if x>thresh else 0,model_4_preds))

# calculating results:
model_4_results=impf.calculate_results(val_labels,model_4_preds_threshed)
model_4_results

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step


{'accuracy': 74.93438320209974,
 'precision': 0.7500996927165142,
 'recall': 0.7493438320209974,
 'f1_score': 0.747278252053036}

In [18]:
# comparing model 4 with baseline
np.array(list(model_4_results.values()))>np.array(list(baseline_results.values()))

array([False, False, False, False])

In [19]:
# comparing model 4 with model 1
np.array(list(model_4_results.values()))>np.array(list(model_1_results.values()))

array([False, False, False, False])

In [20]:
# comparing model 4 with model 2
np.array(list(model_4_results.values()))>np.array(list(model_2_results.values()))

array([ True,  True,  True, False])

In [21]:
# comparing model 4 with model 3
np.array(list(model_4_results.values()))>np.array(list(model_3_results.values()))

array([False, False, False, False])

**Conclusion:**
* Baseline Model, Model 1, Model 3 outperform Model 4.
* Model 4 outpeforms Model 2 some ways 

***-- CONTD IN NEXT NOTEBOOK --***